## Pande Atomic Conv -- Not completed yet --

In [ ]:
%load_ext autoreload
%autoreload 2

### Requires PandeAtomicConv.py

In [ ]:
from PandeAtomicConv import Featurizer
import numpy as np

### Read mols

using RDKit. Complex is combined from ligand and protein.

In [ ]:
from rdkit import Chem

ligand_fname = 'data/1aaq_ligand.sdf'
protein_fname = 'data/1aaq_protein.pdb'

for mol_ligand in Chem.SDMolSupplier(ligand_fname): break
mol_protein = Chem.MolFromPDBFile(protein_fname)
mol_complex = Chem.CombineMols(mol_ligand, mol_protein)

f'{mol_ligand.GetNumAtoms()}, {mol_protein.GetNumAtoms()}, {mol_complex.GetNumAtoms()}'

### Setup featurizers

Calculate neighbor list of size (N, M) (M=12 as the default) in each molecule

In [ ]:
nmax_ligand = 70
nmax_protein = 12000
nmax_complex = nmax_ligand + nmax_protein

featurizer_L = Featurizer(mol_ligand, M=12)
featurizer_P = Featurizer(mol_protein, M=12)
featurizer_C = Featurizer(mol_complex, M=12)

### Calculate convolution

Calculate R (neighbored distance matrix), Z (neighbored atom types) and E (atomic convolution). E is padded to the given size.

In [ ]:
featurizer_L.setConvolution(padding=nmax_ligand)
featurizer_P.setConvolution(padding=nmax_protein)
featurizer_C.setConvolution(padding=nmax_complex)

f'Atomic convolution E.shape: L:{featurizer_L.E.shape}, P:{featurizer_P.E.shape}, C:{featurizer_C.E.shape}'

### Radial pooling layer

Calculate P (Radial pooling layer). radials are given as [1.5, 12.0] step 0.5. $r_s=1.5, \sigma_s = 1.0$ are scalar constants. $\beta_{n_r}$ (scaling) and $b_{n_r}$ (bias) are given as all 1.0 and all 0.0 respectively. These must be elucidated anyway.

In [ ]:
radials = np.arange(1.5, 12.+1e-7, .5)
radials

In [ ]:
r_s, sigma_s = 1.5, 1.
beta = [1.]*len(radials)
bias = [0.]*len(radials)

featurizer_L.setRadialLayer(beta=beta, bias=bias, r_s=r_s, sigma_s=sigma_s, radials=radials)
featurizer_P.setRadialLayer(beta=beta, bias=bias, r_s=r_s, sigma_s=sigma_s, radials=radials)
featurizer_C.setRadialLayer(beta=beta, bias=bias, r_s=r_s, sigma_s=sigma_s, radials=radials)

f'Radial pooling layer P.shape: L:{featurizer_L.P.shape}, P:{featurizer_P.P.shape}, C:{featurizer_C.P.shape}'